## 1. General

This section of for imports, getting an overview of all experiments, and loading the most suitable model.

In [4]:
import sys
import os
import warnings
import logging
from absl import logging as absl_logging

# ✅ Ensure Correct TensorFlow Configuration
os.environ['TFDS_DATA_DIR'] = r"/data/newc6477/VAE/Mean_Beat/All_Physionet/"
os.environ['TF_ENABLE_ONEDNN_OPTS'] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

# ✅ Ensure Correct Working Directory
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print(f"📂 Project Root: {PROJECT_ROOT}")

# ✅ Ensure `src/` is in Python's Path
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

print(f"✅ Updated sys.path:\n{sys.path}")

# ✅ Suppress Warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
absl_logging.set_verbosity(absl_logging.ERROR)

# ✅ Import Modules
try:
    from src.utils.helper import Helper
    from src.evaluate.visualizations import Visualizations
    print("✅ Successfully imported `Helper` and `Visualizations`")
except ModuleNotFoundError as e:
    print(f"❌ Import Error: {e}")
    print("🔍 Check if `src/` has `__init__.py` and is in `sys.path`.")

# ✅ Import Other Required Libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import glob
from neurokit2.signal import signal_smooth

from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split

import ipywidgets as widgets
from matplotlib import pyplot as plt
import seaborn as sns



📂 Project Root: /users/newc6477/VAE/12_Lead_VECG
✅ Updated sys.path:
['/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python310.zip', '/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python3.10', '/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python3.10/lib-dynload', '', '/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python3.10/site-packages', '/users/newc6477/VAE/12_Lead_VECG', '/users/newc6477/VAE/12_Lead_VECG/src', '/users/newc6477/VAE/12_Lead_VECG', '/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python3.10/site-packages/setuptools/_vendor']
✅ Successfully imported `Helper` and `Visualizations`


In [5]:
# The resolution for saving images
DPI = 300

# The source path of the experiments and models
PATH = r"/users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/"

# Some operations take some time in computation.
# Therefore, the stored intermediate results can be used to skip the respective computation.
USE_PRECOMPUTED = True

# 2. Anomaly Detection

## 🔹 Funcs used to train and test

In [6]:
import json
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
import numpy as np
import pandas as pd

def evaluate_models(df_physionet_train, df_physionet_test, hyperparams_list):
    """
    Evaluate a list of models using KNN classification and return a summary table.
    
    Args:
        df_physionet_train (List[pd.DataFrame]): List of train embedding DataFrames (1 per model).
        df_physionet_test (List[pd.DataFrame]): List of test embedding DataFrames (1 per model).
        hyperparams_list (List[dict]): Corresponding list of hyperparameter dictionaries.
    
    Returns:
        pd.DataFrame: Summary table with accuracy, F1 scores, hyperparameters.
    """
    results = []

    for i in range(len(df_physionet_train)):
        print(f"\n🔍 Evaluating model {i+1}/{len(df_physionet_train)}...")

        # Extract hyperparameters and latent dimension
        hparams = hyperparams_list[i]
        latent_dim = hparams.get("latent_dimension", 8)

        # Prepare feature matrices and labels
        X_train = df_physionet_train[i].iloc[:, :latent_dim].values  
        X_test = df_physionet_test[i].iloc[:, :latent_dim].values  
        y_train = np.array(df_physionet_train[i]['diagnostic'].tolist(), dtype=int)  
        y_test = np.array(df_physionet_test[i]['diagnostic'].tolist(), dtype=int)  

        # KNN classification with hyperparameter tuning
        knn = KNeighborsClassifier()
        multi_knn = MultiOutputClassifier(knn, n_jobs=-1)

        param_grid = {"estimator__n_neighbors": [3, 5, 7, 9, 11]}
        grid = GridSearchCV(multi_knn, param_grid, scoring="accuracy", cv=3, n_jobs=-1)
        grid.fit(X_train, y_train)

        best_model = grid.best_estimator_
        y_pred = best_model.predict(X_test)

        # Evaluation metrics
        overall_acc = accuracy_score(y_test, y_pred)
        f1_micro = f1_score(y_test, y_pred, average="micro")
        f1_macro = f1_score(y_test, y_pred, average="macro")
        accuracy_per_label = (y_pred == y_test).mean(axis=0)

        # Print evaluation
        print("✅ Overall Accuracy:", overall_acc)
        print("✅ F1 Score (Macro):", f1_macro)
        print("✅ F1 Score (Micro):", f1_micro)
        print("✅ Accuracy per label:", accuracy_per_label)

        # Classification report
        print("✅ Classification Report:\n")
        print(classification_report(
            y_test, y_pred,
            target_names=[str(i) for i in range(y_test.shape[1])]
        ))

        # Collect results
        results.append({
            "model_index": i,
            "best_k": grid.best_params_["estimator__n_neighbors"],
            "accuracy": overall_acc,
            "f1_micro": f1_micro,
            "f1_macro": f1_macro,
            **hparams
        })

    summary_df = pd.DataFrame(results)
    return summary_df


def extract_hyperparams_from_json(json_path):
    with open(json_path, 'r') as f:
        params = json.load(f)

    # Extract key hyperparameters
    hyperparams = {
        "alpha": params["coefficients"]["alpha"],
        "beta": params["coefficients"]["beta"],
        "gamma": params["coefficients"]["gamma"],
        "latent_dimension": params["latent_dimension"],
        "learning_rate": params["learning_rate"],
        "epochs": params["epochs"]
    }

    return hyperparams

from IPython.display import display
def summarize_by_latent_dimension(results_df):
    unique_dims = results_df["latent_dimension"].unique()
    summary_tables = {}

    for dim in sorted(unique_dims):
        filtered = results_df[results_df["latent_dimension"] == dim]
        sorted_table = filtered.sort_values(by="f1_macro", ascending=False)
        summary_tables[dim] = sorted_table

        print(f"\n📏 Latent Dimension: {dim}")
        display(sorted_table)

    return summary_tables


## 🔹 Lead I Evaluation

In [7]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'I')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/I/2025-04-05_04-50-24/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/I/2025-04-05_05-08-45/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/I/2025-04-05_05-27-05/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/I/2025-04-05_05-45-05/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/I/2025-04-05_06-03-31/model_best.keras
Loaded 5 models successfully.


In [8]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}

In [9]:
df_physionet_train, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='I')
df_physionet_test, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='I')

#df_physionet_train = df_physionet_train[0]
#df_physionet_test = df_physionet_test[0]
#print(type(df_physionet_train))



📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: I
     67/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step
     68/Unknown - 2s 30ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 31ms/step
     68/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 30ms/step
     68/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 28ms/step
     68/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: I
     16/Unknown - 0s 25ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 24ms/step
     16/Unknown - 0s 24ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 23ms/step
     1

In [10]:

results_df = evaluate_models(df_physionet_train,df_physionet_test, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.3377903682719547
✅ F1 Score (Macro): 0.1806151321227034
✅ F1 Score (Micro): 0.4456332548377189
✅ Accuracy per label: [0.93512748 0.90674221 0.99399433 0.9972238  0.98753541 0.97082153
 0.96       0.97983003 0.91252125 0.97382436 0.99484419 0.98090652
 0.9803966  0.98113314 0.75654391 0.96232295 0.98719547 0.99235127
 0.97852691 0.97688385 0.98436261 0.95592068 0.87807365 0.92532578
 0.86515581 0.95535411]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.41      0.13      0.19      1087
           1       0.50      0.37      0.42      1641
           2       0.00      0.00      0.00       106
           3       0.00      0.00      0.00        49
           4       0.78      0.45      0.57       323
           5       0.82      0.60      0.70       974
           6       0.58      0.04      0.08       715
           7       0.00      0.00      0.00       356
           8       0.40      0.02      0.04      1528
  

In [11]:
display(results_df.sort_values(by="f1_macro", ascending=False))

# Set your lead name (e.g., "I", "II")
lead_name = "I"  # change as needed

best_index = results_df.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,11,0.339887,0.448044,0.183855,2.01,0.3,0.2,24,0.002,150
3,3,11,0.336601,0.446229,0.182847,2.01,0.3,0.2,22,0.002,150
1,1,11,0.335241,0.443498,0.181238,2.01,0.3,0.2,18,0.002,150
0,0,11,0.337790,0.445633,0.180615,2.01,0.3,0.2,16,0.002,150
2,2,11,0.330425,0.437927,0.178560,2.01,0.3,0.2,20,0.002,150


## 🔹 Lead II Evaluation


In [12]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'II')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/II/2025-04-05_06-22-30/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/II/2025-04-05_06-40-55/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/II/2025-04-05_06-59-44/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/II/2025-04-05_07-18-20/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/II/2025-04-05_07-37-22/model_best.keras
Loaded 5 models successfully.


In [13]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_II, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='II')
df_physionet_test_II, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='II')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: II
     68/Unknown - 2s 32ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 32ms/step
     66/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 28ms/step
     68/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 28ms/step
     66/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step
     68/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 28ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: II
     15/Unknown - 0s 24ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 23ms/step
     16/Unknown - 0s 24ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 24ms/step
    

In [14]:
results_df_II = evaluate_models(df_physionet_train_II,df_physionet_test_II, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.3243059490084986
✅ F1 Score (Macro): 0.16889962084770593
✅ F1 Score (Micro): 0.4331953999644589
✅ Accuracy per label: [0.93665722 0.90815864 0.99399433 0.9972238  0.98294618 0.95048159
 0.95937677 0.97983003 0.92130312 0.97478754 0.99484419 0.98096317
 0.98033994 0.98107649 0.74600567 0.96215297 0.98787535 0.99235127
 0.9784136  0.97699717 0.98509915 0.95597734 0.87070822 0.91971671
 0.86220963 0.95552408]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.45      0.14      0.21      1087
           1       0.51      0.37      0.43      1641
           2       0.00      0.00      0.00       106
           3       0.00      0.00      0.00        49
           4       0.84      0.08      0.15       323
           5       0.74      0.16      0.26       974
           6       0.48      0.04      0.07       715
           7       0.00      0.00      0.00       356
           8       0.58      0.33      0.42      1528
 

In [15]:
display(results_df_II.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "II"  # change as needed

best_index = results_df_II.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
2,2,11,0.328725,0.440672,0.174394,2.01,0.3,0.2,20,0.002,150
4,4,9,0.328215,0.440366,0.173813,2.01,0.3,0.2,24,0.002,150
3,3,11,0.329292,0.440101,0.172721,2.01,0.3,0.2,22,0.002,150
1,1,11,0.329405,0.440189,0.172419,2.01,0.3,0.2,18,0.002,150
0,0,11,0.324306,0.433195,0.168900,2.01,0.3,0.2,16,0.002,150


## 🔹 Lead III Evaluation


In [16]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'III')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/III/2025-04-05_07-56-16/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/III/2025-04-05_08-14-37/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/III/2025-04-05_08-32-06/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/III/2025-04-05_08-49-45/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/III/2025-04-05_09-07-25/model_best.keras
Loaded 5 models successfully.


In [17]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_III, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='III')
df_physionet_test_III, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='III')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: III
     67/Unknown - 2s 31ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 31ms/step
     66/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 28ms/step
     66/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step
     66/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step
     68/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: III
     15/Unknown - 0s 26ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 24ms/step
     16/Unknown - 0s 24ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 24ms/step
  

In [18]:
results_df_III = evaluate_models(df_physionet_train_III,df_physionet_test_III, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.2628328611898017
✅ F1 Score (Macro): 0.13789923444127108
✅ F1 Score (Micro): 0.36415613561676774
✅ Accuracy per label: [0.93512748 0.90492918 0.99399433 0.9972238  0.98379603 0.95036827
 0.95835694 0.97983003 0.91014164 0.97388102 0.99484419 0.98096317
 0.98045326 0.98096317 0.71025496 0.96220963 0.98572238 0.99235127
 0.9784136  0.97699717 0.98515581 0.95614731 0.84583569 0.91439093
 0.86362606 0.95563739]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.36      0.07      0.12      1087
           1       0.48      0.23      0.31      1641
           2       0.00      0.00      0.00       106
           3       0.00      0.00      0.00        49
           4       0.75      0.17      0.28       323
           5       0.76      0.15      0.25       974
           6       0.28      0.02      0.03       715
           7       0.00      0.00      0.00       356
           8       0.46      0.19      0.27      1528


In [19]:
display(results_df_III.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "III"  # change as needed

best_index = results_df_III.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,9,0.280907,0.388231,0.149312,2.01,0.3,0.2,24,0.002,150
1,1,7,0.272351,0.379819,0.148442,2.01,0.3,0.2,18,0.002,150
2,2,9,0.269292,0.374662,0.142003,2.01,0.3,0.2,20,0.002,150
0,0,9,0.262833,0.364156,0.137899,2.01,0.3,0.2,16,0.002,150
3,3,9,0.260907,0.365856,0.137381,2.01,0.3,0.2,22,0.002,150


## 🔹 Lead aVR Evaluation


In [20]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'aVR')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/aVR/2025-04-05_09-25-08/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/aVR/2025-04-05_09-42-50/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/aVR/2025-04-05_10-00-43/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/aVR/2025-04-05_10-18-15/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/aVR/2025-04-05_10-36-07/model_best.keras
Loaded 5 models successfully.


In [21]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_aVR, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='aVR')
df_physionet_test_aVR, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='aVR')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: aVR
     68/Unknown - 2s 31ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 32ms/step
     68/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 28ms/step
     67/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step
     68/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 30ms/step
     67/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 28ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: aVR
     16/Unknown - 0s 24ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 23ms/step
     15/Unknown - 0s 23ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 23ms/step
  

In [22]:
results_df_aVR = evaluate_models(df_physionet_train_aVR,df_physionet_test_aVR, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.3499150141643059
✅ F1 Score (Macro): 0.16400236954644554
✅ F1 Score (Micro): 0.4530347372439242
✅ Accuracy per label: [0.93433428 0.90538244 0.99399433 0.9972238  0.98328612 0.96543909
 0.96016997 0.97977337 0.91195467 0.97410765 0.99484419 0.98096317
 0.9803966  0.98107649 0.76532578 0.96220963 0.9882153  0.99235127
 0.97852691 0.97699717 0.98509915 0.95620397 0.88266289 0.93308782
 0.86379603 0.95484419]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.38      0.10      0.16      1087
           1       0.49      0.34      0.40      1641
           2       0.00      0.00      0.00       106
           3       0.00      0.00      0.00        49
           4       0.91      0.10      0.17       323
           5       0.81      0.49      0.61       974
           6       0.59      0.06      0.10       715
           7       0.00      0.00      0.00       356
           8       0.41      0.04      0.07      1528
 

In [23]:
display(results_df_aVR.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "aVR"  # change as needed

best_index = results_df_aVR.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,11,0.352465,0.461021,0.170650,2.01,0.3,0.2,24,0.002,150
1,1,11,0.345949,0.453098,0.167595,2.01,0.3,0.2,18,0.002,150
3,3,11,0.351898,0.457769,0.166849,2.01,0.3,0.2,22,0.002,150
2,2,11,0.344873,0.450616,0.165058,2.01,0.3,0.2,20,0.002,150
0,0,11,0.349915,0.453035,0.164002,2.01,0.3,0.2,16,0.002,150


## 🔹 Lead aVL Evaluation


In [24]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'aVL')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/aVL/2025-04-04_17-25-22/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/aVL/2025-04-04_17-45-10/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/aVL/2025-04-04_18-04-54/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/aVL/2025-04-04_18-24-47/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/aVL/2025-04-04_18-44-27/model_best.keras
Loaded 5 models successfully.


In [25]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_aVL, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='aVL')
df_physionet_test_aVL, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='aVL')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: aVL
     68/Unknown - 2s 32ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 32ms/step
     68/Unknown - 2s 30ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 30ms/step
     66/Unknown - 2s 30ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 30ms/step
     68/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 30ms/step
     66/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: aVL
     16/Unknown - 0s 24ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 24ms/step
     14/Unknown - 0s 27ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 25ms/step
  

In [26]:
results_df_aVL = evaluate_models(df_physionet_train_aVL,df_physionet_test_aVL, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.2673654390934844
✅ F1 Score (Macro): 0.15151176098357164
✅ F1 Score (Micro): 0.37256375838926176
✅ Accuracy per label: [0.93456091 0.90606232 0.99399433 0.9972238  0.98628895 0.96181303
 0.9592068  0.97983003 0.91280453 0.97365439 0.99478754 0.98096317
 0.98028329 0.98101983 0.7207932  0.96226629 0.98634561 0.99235127
 0.97835694 0.97705382 0.98521246 0.95569405 0.84220963 0.91552408
 0.8623796  0.95512748]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.32      0.06      0.10      1087
           1       0.49      0.20      0.29      1641
           2       0.00      0.00      0.00       106
           3       0.00      0.00      0.00        49
           4       0.77      0.36      0.49       323
           5       0.79      0.42      0.55       974
           6       0.41      0.02      0.03       715
           7       0.00      0.00      0.00       356
           8       0.49      0.12      0.20      1528


In [27]:
display(results_df_aVL.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "aVF"  # change as needed

best_index = results_df_aVL.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
1,1,7,0.260793,0.367583,0.153793,2.01,0.3,0.2,18,0.002,150
3,3,7,0.265269,0.369024,0.153037,2.01,0.3,0.2,22,0.002,150
0,0,9,0.267365,0.372564,0.151512,2.01,0.3,0.2,16,0.002,150
2,2,7,0.260793,0.362724,0.148770,2.01,0.3,0.2,20,0.002,150
4,4,9,0.264476,0.366839,0.146559,2.01,0.3,0.2,24,0.002,150


## aVF

In [28]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'aVF')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/aVF/2025-04-04_19-04-23/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/aVF/2025-04-04_19-24-24/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/aVF/2025-04-04_19-43-27/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/aVF/2025-04-04_20-03-52/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/aVF/2025-04-04_20-24-46/model_best.keras
Loaded 5 models successfully.


In [29]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_aVF, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='aVF')
df_physionet_test_aVF, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='aVF')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: aVF
     67/Unknown - 2s 32ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 32ms/step
     68/Unknown - 2s 30ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 30ms/step
     66/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step
     67/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step
     68/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: aVF
     15/Unknown - 0s 26ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 25ms/step
     15/Unknown - 0s 27ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 26ms/step
  

In [30]:
results_df_aVF = evaluate_models(df_physionet_train_aVF,df_physionet_test_aVF, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.2927478753541076
✅ F1 Score (Macro): 0.1487345719841972
✅ F1 Score (Micro): 0.3987790366806353
✅ Accuracy per label: [0.93518414 0.90674221 0.99399433 0.9972238  0.98226629 0.94481586
 0.95954674 0.97983003 0.91609065 0.97444759 0.99484419 0.98096317
 0.9803966  0.98107649 0.72577904 0.96220963 0.98815864 0.99235127
 0.97858357 0.97688385 0.98509915 0.95580737 0.85558074 0.918017
 0.86192635 0.95535411]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.40      0.10      0.17      1087
           1       0.50      0.32      0.39      1641
           2       0.00      0.00      0.00       106
           3       0.00      0.00      0.00        49
           4       0.69      0.06      0.10       323
           5       0.50      0.02      0.03       974
           6       0.51      0.03      0.06       715
           7       0.00      0.00      0.00       356
           8       0.52      0.34      0.41      1528
    

In [31]:
display(results_df_aVF.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "aVF"  # change as needed

best_index = results_df_aVF.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
0,0,9,0.292748,0.398779,0.148735,2.01,0.3,0.2,16,0.002,150
4,4,11,0.290368,0.398791,0.147237,2.01,0.3,0.2,24,0.002,150
3,3,11,0.291615,0.396733,0.146009,2.01,0.3,0.2,22,0.002,150
2,2,11,0.298300,0.402467,0.145680,2.01,0.3,0.2,20,0.002,150
1,1,11,0.293371,0.398888,0.145013,2.01,0.3,0.2,18,0.002,150


## 🔹 Lead V1 Evaluation

In [32]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V1')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V1/2025-04-04_20-44-47/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V1/2025-04-04_21-05-27/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V1/2025-04-04_21-26-30/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V1/2025-04-04_21-47-23/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V1/2025-04-04_22-07-36/model_best.keras
Loaded 5 models successfully.


In [33]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V1, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V1')
df_physionet_test_V1, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V1')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V1
     67/Unknown - 2s 32ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 32ms/step
     66/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step
     66/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step
     66/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 28ms/step
     67/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V1
     15/Unknown - 0s 26ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 25ms/step
     14/Unknown - 0s 24ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 23ms/step
    

In [34]:
results_df_V1 = evaluate_models(df_physionet_train_V1,df_physionet_test_V1, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.3317847025495751
✅ F1 Score (Macro): 0.1615952189061673
✅ F1 Score (Micro): 0.44066645774574326
✅ Accuracy per label: [0.93626062 0.90300283 0.99399433 0.9972238  0.98730878 0.97779037
 0.95988669 0.9792068  0.91223796 0.97393768 0.99484419 0.98090652
 0.9803966  0.98107649 0.75966006 0.96232295 0.98668555 0.99235127
 0.97858357 0.97699717 0.98509915 0.95631728 0.88107649 0.92719547
 0.86634561 0.95580737]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.40      0.07      0.11      1087
           1       0.44      0.15      0.22      1641
           2       0.00      0.00      0.00       106
           3       0.00      0.00      0.00        49
           4       0.79      0.41      0.54       323
           5       0.81      0.78      0.79       974
           6       0.63      0.02      0.05       715
           7       0.40      0.06      0.11       356
           8       0.32      0.01      0.02      1528
 

In [35]:
display(results_df_V1.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V1"  # change as needed

best_index = results_df_V1.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,11,0.342323,0.452167,0.165008,2.01,0.3,0.2,24,0.002,150
1,1,9,0.334108,0.445675,0.163855,2.01,0.3,0.2,18,0.002,150
3,3,11,0.334731,0.442085,0.162110,2.01,0.3,0.2,22,0.002,150
2,2,11,0.335184,0.444549,0.161857,2.01,0.3,0.2,20,0.002,150
0,0,11,0.331785,0.440666,0.161595,2.01,0.3,0.2,16,0.002,150


## 🔹 Lead V2 Evaluation


In [36]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V2')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V2/2025-04-04_22-28-33/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V2/2025-04-04_22-49-58/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V2/2025-04-04_23-10-11/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V2/2025-04-04_23-30-43/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V2/2025-04-04_23-51-26/model_best.keras
Loaded 5 models successfully.


In [37]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V2, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V2')
df_physionet_test_V2, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V2')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V2
     66/Unknown - 2s 31ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 31ms/step
     66/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 28ms/step
     67/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 28ms/step
     68/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 28ms/step
     68/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 28ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V2
     17/Unknown - 0s 24ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 24ms/step
     15/Unknown - 0s 24ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 24ms/step
    

In [38]:
results_df_V2 = evaluate_models(df_physionet_train_V2,df_physionet_test_V2, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.3240226628895184
✅ F1 Score (Macro): 0.15115342250617037
✅ F1 Score (Micro): 0.42738929279577
✅ Accuracy per label: [0.93456091 0.90447592 0.99399433 0.9972238  0.98844193 0.96498584
 0.95926346 0.97988669 0.9121813  0.97393768 0.99484419 0.98096317
 0.9803966  0.98107649 0.75104816 0.96232295 0.98594901 0.99235127
 0.97852691 0.97699717 0.98509915 0.95614731 0.88441926 0.93223796
 0.86577904 0.95575071]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.31      0.05      0.09      1087
           1       0.46      0.15      0.23      1641
           2       0.00      0.00      0.00       106
           3       0.00      0.00      0.00        49
           4       0.76      0.54      0.63       323
           5       0.78      0.51      0.62       974
           6       0.39      0.01      0.02       715
           7       0.60      0.01      0.02       356
           8       0.36      0.02      0.04      1528
   

In [39]:
display(results_df_V2.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V2"  # change as needed

best_index = results_df_V2.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,11,0.331161,0.440347,0.154468,2.01,0.3,0.2,24,0.002,150
3,3,11,0.330255,0.436339,0.152116,2.01,0.3,0.2,22,0.002,150
0,0,11,0.324023,0.427389,0.151153,2.01,0.3,0.2,16,0.002,150
1,1,11,0.329462,0.435579,0.150899,2.01,0.3,0.2,18,0.002,150
2,2,11,0.328329,0.432442,0.148616,2.01,0.3,0.2,20,0.002,150


## 🔹 Lead V3 Evaluation


In [40]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V3')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V3/2025-04-05_00-12-16/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V3/2025-04-05_00-32-15/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V3/2025-04-05_00-53-49/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V3/2025-04-05_01-14-34/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V3/2025-04-05_01-35-56/model_best.keras
Loaded 5 models successfully.


In [41]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V3, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V3')
df_physionet_test_V3, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V3')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V3
     66/Unknown - 2s 32ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 32ms/step
     67/Unknown - 2s 30ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 30ms/step
     67/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 30ms/step
     67/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step
     68/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V3
     16/Unknown - 0s 24ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 24ms/step
     16/Unknown - 0s 26ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 25ms/step
    

In [42]:
results_df_V3 = evaluate_models(df_physionet_train_V3,df_physionet_test_V3, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.33592067988668556
✅ F1 Score (Macro): 0.14895616925375393
✅ F1 Score (Micro): 0.43769202728740303
✅ Accuracy per label: [0.93478754 0.90566572 0.99399433 0.9972238  0.98708215 0.95688385
 0.95954674 0.97983003 0.9129745  0.97399433 0.99484419 0.98096317
 0.9803966  0.98107649 0.76300283 0.96232295 0.98691218 0.99235127
 0.97835694 0.97699717 0.98509915 0.95620397 0.8905949  0.92430595
 0.86538244 0.95563739]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.32      0.05      0.09      1087
           1       0.48      0.20      0.28      1641
           2       0.00      0.00      0.00       106
           3       0.00      0.00      0.00        49
           4       0.75      0.44      0.55       323
           5       0.80      0.29      0.43       974
           6       0.56      0.01      0.01       715
           7       0.00      0.00      0.00       356
           8       0.46      0.03      0.06      1528

In [43]:
display(results_df_V3.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V3"  # change as needed

best_index = results_df_V3.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
3,3,11,0.346572,0.451711,0.158998,2.01,0.3,0.2,22,0.002,150
1,1,11,0.345212,0.447690,0.158172,2.01,0.3,0.2,18,0.002,150
2,2,11,0.342833,0.445406,0.156060,2.01,0.3,0.2,20,0.002,150
4,4,11,0.337337,0.441417,0.155935,2.01,0.3,0.2,24,0.002,150
0,0,11,0.335921,0.437692,0.148956,2.01,0.3,0.2,16,0.002,150


## 🔹 Lead V4 Evaluation


In [44]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V4')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V4/2025-04-05_01-56-01/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V4/2025-04-05_02-14-54/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V4/2025-04-05_02-34-09/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V4/2025-04-05_02-53-14/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V4/2025-04-05_03-12-29/model_best.keras
Loaded 5 models successfully.


In [45]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V4, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V4')
df_physionet_test_V4, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V4')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V4
     66/Unknown - 2s 31ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 31ms/step
     67/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 30ms/step
     66/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step
     66/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step
     66/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V4
     15/Unknown - 0s 25ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 24ms/step
     15/Unknown - 0s 25ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 23ms/step
    

In [46]:
results_df_V4 = evaluate_models(df_physionet_train_V4,df_physionet_test_V4, hyperparams_list)


🔍 Evaluating model 1/5...
✅ Overall Accuracy: 0.34402266288951844
✅ F1 Score (Macro): 0.14348840837509408
✅ F1 Score (Micro): 0.43741059585424846
✅ Accuracy per label: [0.93529745 0.90379603 0.99399433 0.9972238  0.98311615 0.95303116
 0.95988669 0.97983003 0.91155807 0.97388102 0.99484419 0.98096317
 0.9803966  0.98107649 0.77263456 0.96232295 0.98674221 0.99235127
 0.97847025 0.97699717 0.98509915 0.95620397 0.89206799 0.92294618
 0.86385269 0.95586402]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.38      0.08      0.14      1087
           1       0.47      0.23      0.31      1641
           2       0.00      0.00      0.00       106
           3       0.00      0.00      0.00        49
           4       0.61      0.22      0.32       323
           5       0.80      0.20      0.32       974
           6       0.73      0.02      0.03       715
           7       0.00      0.00      0.00       356
           8       0.36   

In [47]:
display(results_df_V4.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V4"  # change as needed

best_index = results_df_V4.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
1,1,11,0.346176,0.440626,0.144712,2.01,0.3,0.2,18,0.002,150
2,2,11,0.343116,0.436603,0.144155,2.01,0.3,0.2,20,0.002,150
4,4,11,0.338187,0.431283,0.143859,2.01,0.3,0.2,24,0.002,150
3,3,11,0.340057,0.435556,0.143687,2.01,0.3,0.2,22,0.002,150
0,0,11,0.344023,0.437411,0.143488,2.01,0.3,0.2,16,0.002,150


## 🔹 Lead V5 Evaluation


In [48]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V5')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V5/2025-04-05_03-31-38/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V5/2025-04-05_03-50-41/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V5/2025-04-05_04-09-43/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V5/2025-04-05_04-29-35/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V5/2025-04-05_13-15-45/model_best.keras
Loaded 5 models successfully.


In [49]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V5, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V5')
df_physionet_test_V5, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V5')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V5
     67/Unknown - 2s 31ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 31ms/step
     68/Unknown - 2s 30ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 30ms/step
     68/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step
     67/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step
     68/Unknown - 2s 29ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 29ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V5
     16/Unknown - 0s 23ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 23ms/step
     17/Unknown - 0s 24ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 24ms/step
    

In [50]:
results_df_V5 = evaluate_models(df_physionet_train_V5,df_physionet_test_V5, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.3453257790368272
✅ F1 Score (Macro): 0.1558293154349513
✅ F1 Score (Micro): 0.44324883600620796
✅ Accuracy per label: [0.93711048 0.90458924 0.99399433 0.9972238  0.98441926 0.95835694
 0.95977337 0.97983003 0.91246459 0.97399433 0.99484419 0.98096317
 0.9803966  0.98107649 0.77382436 0.96232295 0.98634561 0.99235127
 0.97807365 0.97694051 0.98509915 0.95620397 0.88696884 0.92419263
 0.86407932 0.95507082]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.45      0.10      0.16      1087
           1       0.47      0.25      0.33      1641
           2       0.00      0.00      0.00       106
           3       0.00      0.00      0.00        49
           4       0.77      0.21      0.33       323
           5       0.79      0.33      0.47       974
           6       0.65      0.02      0.03       715
           7       0.00      0.00      0.00       356
           8       0.44      0.04      0.08      1528
 

In [51]:
display(results_df_V5.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V5"  # change as needed

best_index = results_df_V5.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
2,2,11,0.345382,0.448441,0.161552,2.01,0.3,0.2,20,0.002,150
1,1,11,0.346176,0.444187,0.157396,2.01,0.3,0.2,18,0.002,150
3,3,11,0.344929,0.441398,0.156927,2.01,0.3,0.2,22,0.002,150
0,0,11,0.345326,0.443249,0.155829,2.01,0.3,0.2,16,0.002,150
4,4,11,0.340453,0.435283,0.151665,2.01,0.3,0.2,24,0.002,150


## 🔹 Lead V6 Evaluation


In [52]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V6')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V6/2025-04-05_13-18-31/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V6/2025-04-05_13-36-33/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V6/2025-04-05_13-54-02/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V6/2025-04-05_14-13-43/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/mean_all/test_is_split1/V6/2025-04-05_14-35-39/model_best.keras
Loaded 5 models successfully.


In [53]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V6, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V6')
df_physionet_test_V6, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V6')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V6
     66/Unknown - 2s 30ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 30ms/step
     68/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 28ms/step
     67/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 28ms/step
     68/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 28ms/step
     68/Unknown - 2s 28ms/step✅ Generator exhausted normally.
69/69 [==============================] - 2s 28ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V6
     16/Unknown - 0s 25ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 23ms/step
     16/Unknown - 0s 24ms/step✅ Generator exhausted normally.
18/18 [==============================] - 0s 24ms/step
    

In [54]:
results_df_V6 = evaluate_models(df_physionet_train_V6,df_physionet_test_V6, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.3311614730878187
✅ F1 Score (Macro): 0.16272224632307733
✅ F1 Score (Micro): 0.430785204412719
✅ Accuracy per label: [0.93552408 0.90509915 0.99399433 0.9972238  0.98866856 0.96209632
 0.95983003 0.97977337 0.9129745  0.97331445 0.99484419 0.98096317
 0.98033994 0.98107649 0.76617564 0.96232295 0.98696884 0.99235127
 0.97824363 0.97699717 0.98492918 0.95614731 0.87937677 0.9094051
 0.86356941 0.95535411]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.41      0.10      0.17      1087
           1       0.48      0.26      0.34      1641
           2       0.00      0.00      0.00       106
           3       0.00      0.00      0.00        49
           4       0.86      0.46      0.60       323
           5       0.80      0.42      0.55       974
           6       0.62      0.02      0.04       715
           7       0.00      0.00      0.00       356
           8       0.47      0.04      0.08      1528
   

In [55]:
display(results_df_V6.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V6"  # change as needed

best_index = results_df_V6.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
1,1,11,0.331501,0.434151,0.165562,2.01,0.3,0.2,18,0.002,150
2,2,11,0.331218,0.430616,0.164414,2.01,0.3,0.2,20,0.002,150
0,0,11,0.331161,0.430785,0.162722,2.01,0.3,0.2,16,0.002,150
3,3,11,0.331218,0.431548,0.162556,2.01,0.3,0.2,22,0.002,150
4,4,11,0.329972,0.430302,0.161699,2.01,0.3,0.2,24,0.002,150
